#### Data Ingestion & Curation


In [0]:
%python
storage_account_name = "formula1rawdata"
container_name = "f1rawdata"
mount_point = "/mnt/formula1datalake"
access_key = "0gmSUiLIK9ocU12jgP9rt5g3w8oTi/Z6BVvA3AX/Ku7ZBKbm81pZTJTki9dB7JwVLM0zOcrLk/yr+AStxbD2Ow=="


In [0]:
%python
# Unmount first if it already exists
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

# Mount to the Azure container
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = mount_point,
    extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)


/mnt/formula1datalake has been unmounted.


True

In [0]:
%python
spark.conf.set(
  "fs.azure.account.key.formula1rawdata.dfs.core.windows.net",
  "0gmSUiLIK9ocU12jgP9rt5g3w8oTi/Z6BVvA3AX/Ku7ZBKbm81pZTJTki9dB7JwVLM0zOcrLk/yr+AStxbD2Ow=="
)

In [0]:
%python
display(dbutils.fs.ls("/mnt/formula1datalake"))


path,name,size,modificationTime
dbfs:/mnt/formula1datalake/circuits.csv,circuits.csv,10044,1761067663000
dbfs:/mnt/formula1datalake/constructors.json,constructors.json,30415,1761067663000
dbfs:/mnt/formula1datalake/drivers.json,drivers.json,180812,1761067663000
dbfs:/mnt/formula1datalake/lap_times/,lap_times/,0,0
dbfs:/mnt/formula1datalake/pit_stops.json,pit_stops.json,1369387,1761067663000
dbfs:/mnt/formula1datalake/qualifying/,qualifying/,0,0
dbfs:/mnt/formula1datalake/races.csv,races.csv,116847,1761067663000
dbfs:/mnt/formula1datalake/results.json,results.json,7165641,1761067665000


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_raw
-- dbfs:/user/hive/warehouse/f1_raw.db

In [0]:
CREATE DATABASE IF NOT EXISTS f1_presentation
LOCATION '/mnt/formula1dl/presentation'

In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.circuits;
CREATE TABLE IF NOT EXISTS f1_raw.circuits (
  circuitId INT,
  circuitRef STRING,
  name STRING,
  location STRING,
  country STRING,
  lat DOUBLE,
  lng DOUBLE,
  alt INT,
  url STRING
)
USING csv
OPTIONS (path "/mnt/formula1datalake/circuits.csv", header "true");


In [0]:
%sql
SELECT * FROM f1_raw.circuits LIMIT 10;


circuitId,circuitRef,name,location,country,lat,lng,alt,url
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,http://en.wikipedia.org/wiki/Sepang_International_Circuit
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,http://en.wikipedia.org/wiki/Istanbul_Park
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,http://en.wikipedia.org/wiki/Circuit_Gilles_Villeneuve
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,http://en.wikipedia.org/wiki/Circuit_de_Nevers_Magny-Cours
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,http://en.wikipedia.org/wiki/Silverstone_Circuit
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,http://en.wikipedia.org/wiki/Hockenheimring


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.races;
CREATE TABLE IF NOT EXISTS f1_raw.races (
  raceId INT,
  year INT,
  round INT,
  circuitId INT,
  name STRING,
  date DATE,
  time STRING,
  url STRING
)
USING csv
OPTIONS (path "/mnt/formula1datalake/races.csv", header "true");


In [0]:
%sql
SELECT * FROM f1_raw.races LIMIT 10;


raceId,year,round,circuitId,name,date,time,url
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix
6,2009,6,6,Monaco Grand Prix,2009-05-24,12:00:00,http://en.wikipedia.org/wiki/2009_Monaco_Grand_Prix
7,2009,7,5,Turkish Grand Prix,2009-06-07,12:00:00,http://en.wikipedia.org/wiki/2009_Turkish_Grand_Prix
8,2009,8,9,British Grand Prix,2009-06-21,12:00:00,http://en.wikipedia.org/wiki/2009_British_Grand_Prix
9,2009,9,20,German Grand Prix,2009-07-12,12:00:00,http://en.wikipedia.org/wiki/2009_German_Grand_Prix
10,2009,10,11,Hungarian Grand Prix,2009-07-26,12:00:00,http://en.wikipedia.org/wiki/2009_Hungarian_Grand_Prix


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.constructors;
CREATE TABLE IF NOT EXISTS f1_raw.constructors (
  constructorId INT,
  constructorRef STRING,
  name STRING,
  nationality STRING,
  url STRING
)
USING json
OPTIONS (path "/mnt/formula1datalake/constructors.json");


In [0]:
%sql
SELECT * FROM f1_raw.constructors LIMIT 10;


constructorId,constructorRef,name,nationality,url
1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering
4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formula_One
5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari
7,toyota,Toyota,Japanese,http://en.wikipedia.org/wiki/Toyota_Racing
8,super_aguri,Super Aguri,Japanese,http://en.wikipedia.org/wiki/Super_Aguri_F1
9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing
10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Force_India


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.drivers;
CREATE TABLE IF NOT EXISTS f1_raw.drivers (
  driverId INT,
  driverRef STRING,
  number INT,
  code STRING,
  name STRUCT<forename: STRING, surname: STRING>,
  dob DATE,
  nationality STRING,
  url STRING
)
USING json
OPTIONS (path "/mnt/formula1datalake/drivers.json");

In [0]:
%sql
SELECT * FROM f1_raw.drivers LIMIT 10;


driverId,driverRef,number,code,name,dob,nationality,url
1,hamilton,44,HAM,"List(Lewis, Hamilton)",1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,heidfeld,null,HEI,"List(Nick, Heidfeld)",1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
3,rosberg,6,ROS,"List(Nico, Rosberg)",1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
4,alonso,14,ALO,"List(Fernando, Alonso)",1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
5,kovalainen,null,KOV,"List(Heikki, Kovalainen)",1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
6,nakajima,null,NAK,"List(Kazuki, Nakajima)",1985-01-11,Japanese,http://en.wikipedia.org/wiki/Kazuki_Nakajima
7,bourdais,null,BOU,"List(Sébastien, Bourdais)",1979-02-28,French,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais
8,raikkonen,7,RAI,"List(Kimi, Räikkönen)",1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen
9,kubica,88,KUB,"List(Robert, Kubica)",1984-12-07,Polish,http://en.wikipedia.org/wiki/Robert_Kubica
10,glock,null,GLO,"List(Timo, Glock)",1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.results;
CREATE TABLE IF NOT EXISTS f1_raw.results (
  resultId INT,
  raceId INT,
  driverId INT,
  constructorId INT,
  number INT,
  grid INT,
  position INT,
  positionText STRING,
  positionOrder INT,
  points FLOAT,
  laps INT,
  time STRING,
  milliseconds INT,
  fastestLap INT,
  rank INT,
  fastestLapTime STRING,
  fastestLapSpeed FLOAT,
  statusId STRING
)
USING json
OPTIONS (path "/mnt/formula1datalake/results.json");

In [0]:
%sql
SELECT * FROM f1_raw.results LIMIT 10;


resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.3,1
2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
6,18,6,3,8,13,6,6,6,3.0,57,\N,null,50,14,1:29.639,212.974,11
7,18,7,5,14,17,7,7,7,2.0,55,\N,null,22,12,1:29.534,213.224,5
8,18,8,6,1,15,8,8,8,1.0,53,\N,null,20,4,1:27.903,217.18,5
9,18,9,2,4,2,null,R,9,0.0,47,\N,null,15,9,1:28.753,215.1,4
10,18,10,7,12,18,null,R,10,0.0,43,\N,null,23,13,1:29.558,213.166,3


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.pit_stops;
CREATE TABLE IF NOT EXISTS f1_raw.pit_stops (
  driverId INT,
  duration STRING,
  lap INT,
  milliseconds INT,
  raceId INT,
  stop INT,
  time STRING
)
USING json
OPTIONS (path "/mnt/formula1datalake/pit_stops.json", multiLine = true);

In [0]:
%sql
SELECT * FROM f1_raw.pit_stops LIMIT 10;


driverId,duration,lap,milliseconds,raceId,stop,time
153,26.898,1,26898,841,1,17:05:23
30,25.021,1,25021,841,1,17:05:52
17,23.426,11,23426,841,1,17:20:48
4,23.251,12,23251,841,1,17:22:34
13,23.842,13,23842,841,1,17:24:10
22,23.643,13,23643,841,1,17:24:29
20,22.603,14,22603,841,1,17:25:17
814,24.863,14,24863,841,1,17:26:03
816,25.259,14,25259,841,1,17:26:50
67,25.342,15,25342,841,1,17:27:34


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.lap_times;
CREATE TABLE IF NOT EXISTS f1_raw.lap_times (
  receId INT,
  driverId INT,
  lap INT,
  position INT,
  time STRING,
  milliseconds INT
)
USING csv
OPTIONS (path "/mnt/formula1datalake/lap_times");

In [0]:
%sql
SELECT * FROM f1_raw.lap_times LIMIT 10;


receId,driverId,lap,position,time,milliseconds
841,20,1,1,1:38.109,98109
841,20,2,1,1:33.006,93006
841,20,3,1,1:32.713,92713
841,20,4,1,1:32.803,92803
841,20,5,1,1:32.342,92342
841,20,6,1,1:32.605,92605
841,20,7,1,1:32.502,92502
841,20,8,1,1:32.537,92537
841,20,9,1,1:33.240,93240
841,20,10,1,1:32.572,92572


In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.qualifying;
CREATE TABLE IF NOT EXISTS f1_raw.qualifying (
  constructorId INT,
  driverId INT,
  position INT,
  q1 STRING,
  q2 STRING,
  q3 STRING,
  qualifingId INT,
  raceId INT
)
USING json
OPTIONS (path "/mnt/formula1datalake/qualifying", multiLine = true);

In [0]:
SELECT * FROM f1_raw.qualifying LIMIT 10;


constructorId,driverId,position,q1,q2,q3,qualifingId,raceId
1,1,1,1:26.572,1:25.187,1:26.714,null,18
2,9,2,1:26.103,1:25.315,1:26.869,null,18
1,5,3,1:25.664,1:25.452,1:27.079,null,18
6,13,4,1:25.994,1:25.691,1:27.178,null,18
2,2,5,1:25.960,1:25.518,1:27.236,null,18
7,15,6,1:26.427,1:26.101,1:28.527,null,18
3,3,7,1:26.295,1:26.059,1:28.687,null,18
9,14,8,1:26.381,1:26.063,1:29.041,null,18
7,10,9,1:26.919,1:26.164,1:29.593,null,18
5,20,10,1:26.702,1:25.842,\N,null,18


In [0]:
%python
df = spark.table("f1_raw.circuits")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/circuits")

In [0]:
%python
df = spark.table("f1_raw.races")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/races")

In [0]:
%python
df = spark.table("f1_raw.constructors")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/constructors")

In [0]:
%python
df = spark.table("f1_raw.drivers")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/drivers")

In [0]:
%python
df = spark.table("f1_raw.results")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/results")

In [0]:
%python
df = spark.table("f1_raw.pit_stops")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/pit_stops")

In [0]:
%python
df = spark.table("f1_raw.lap_times")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/lap_times")

In [0]:
%python
df = spark.table("f1_raw.qualifying")
df.write \
  .mode("overwrite") \
  .format("parquet") \
  .save("abfss://f1processed@formula1rawdata.dfs.core.windows.net/qualifying")

#### Data Analysis


In [0]:
use f1_raw;
drop table if exists f1_presentation.calc_race_results;
create table f1_presentation.calc_race_results
using parquet 
as 
select races.year,
constructors.name as team_name,
drivers.name as driver_name,
results.position,
results.points,
11-results.position as calc_points
from f1_raw.results  
join f1_raw.drivers on (results.driverId=drivers.driverId)
join f1_raw.constructors on (results.constructorId=constructors.constructorId)
join f1_raw.races on (results.raceId=races.raceId)
where results.position <= 10

In [0]:
select * from f1_presentation.calc_race_results

year,team_name,driver_name,position,points,calc_points
1975,Ensign,"List(Gijs, van Lennep)",10,0.0,1
1975,Ferrari,"List(Niki, Lauda)",1,9.0,10
1975,Hesketh,"List(James, Hunt)",2,6.0,9
1975,McLaren,"List(Jochen, Mass)",3,4.0,8
1975,McLaren,"List(Emerson, Fittipaldi)",4,3.0,7
1975,Parnelli,"List(Mario, Andretti)",5,2.0,6
1975,Tyrrell,"List(Patrick, Depailler)",6,1.0,5
1975,Embassy Hill,"List(Tony, Brise)",7,0.0,4
1975,Shadow-Ford,"List(Jean-Pierre, Jarier)",8,0.0,3
1975,Tyrrell,"List(Jody, Scheckter)",9,0.0,2


In [0]:
%skip
%python
html = """<h1 style="color:Black;text-align:center;font-family:Ariel;">Report on Dominant Formula 1 Drivers </h1>"""
displayHTML(html)

Report on Dominant Formula 1 Drivers

In [0]:
select concat(driver_name.forename, ' ', driver_name.surname) as driver_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
group by driver_name
having count(1) >= 50
order by avg_points desc


driver_name,total_races,total_points,avg_points
Alain Prost,138,1156,8.376811594202898
Ayrton Senna,104,869,8.35576923076923
Jackie Stewart,61,507,8.311475409836065
Lewis Hamilton,232,1903,8.202586206896552
Michael Schumacher,226,1796,7.946902654867257
Nigel Mansell,90,713,7.9222222222222225
Niki Lauda,81,639,7.888888888888889
Sebastian Vettel,203,1565,7.70935960591133
Damon Hill,70,520,7.428571428571429
Juan Pablo Montoya,58,427,7.362068965517241


In [0]:
create or replace temp view vw_dominant_drivers
as
select concat(driver_name.forename, ' ', driver_name.surname) as driver_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points,
rank() over(order by sum(calc_points) / count(1) desc) as driver_rank
from f1_presentation.calc_race_results
group by driver_name
having count(1) >= 50
order by avg_points desc


In [0]:
select to_date(year, 'yyyy') as year, 
concat(driver_name.forename, ' ', driver_name.surname) as driver_name,
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where concat(driver_name.forename, ' ', driver_name.surname) in (select driver_name from vw_dominant_drivers where driver_rank <=10 )
group by year, driver_name
order by year,avg_points desc

year,driver_name,total_races,total_points,avg_points
1965-01-01,Jackie Stewart,7,56,8.0
1966-01-01,Jackie Stewart,3,23,7.666666666666667
1967-01-01,Jackie Stewart,2,17,8.5
1968-01-01,Jackie Stewart,8,59,7.375
1969-01-01,Jackie Stewart,8,76,9.5
1970-01-01,Jackie Stewart,5,38,7.6
1971-01-01,Jackie Stewart,8,75,9.375
1972-01-01,Jackie Stewart,7,60,8.571428571428571
1972-01-01,Niki Lauda,3,7,2.3333333333333335
1973-01-01,Jackie Stewart,13,103,7.923076923076923


Databricks visualization. Run in Databricks to view.

In [0]:
select to_date(year, 'yyyy') as year, 
concat(driver_name.forename, ' ', driver_name.surname) as driver_name,
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where concat(driver_name.forename, ' ', driver_name.surname) in (select driver_name from vw_dominant_drivers where driver_rank <=10 )
group by year, driver_name
order by year,avg_points desc

year,driver_name,total_races,total_points,avg_points
1965-01-01,Jackie Stewart,7,56,8.0
1966-01-01,Jackie Stewart,3,23,7.666666666666667
1967-01-01,Jackie Stewart,2,17,8.5
1968-01-01,Jackie Stewart,8,59,7.375
1969-01-01,Jackie Stewart,8,76,9.5
1970-01-01,Jackie Stewart,5,38,7.6
1971-01-01,Jackie Stewart,8,75,9.375
1972-01-01,Jackie Stewart,7,60,8.571428571428571
1972-01-01,Niki Lauda,3,7,2.3333333333333335
1973-01-01,Jackie Stewart,13,103,7.923076923076923


Databricks visualization. Run in Databricks to view.

In [0]:
select  to_date(year, 'yyyy') as year, 
concat(driver_name.forename, ' ', driver_name.surname) as driver_name,
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where concat(driver_name.forename, ' ', driver_name.surname) in (select driver_name from vw_dominant_drivers where driver_rank <=10 )
group by year, driver_name
order by year,avg_points desc

year,driver_name,total_races,total_points,avg_points
1965-01-01,Jackie Stewart,7,56,8.0
1966-01-01,Jackie Stewart,3,23,7.666666666666667
1967-01-01,Jackie Stewart,2,17,8.5
1968-01-01,Jackie Stewart,8,59,7.375
1969-01-01,Jackie Stewart,8,76,9.5
1970-01-01,Jackie Stewart,5,38,7.6
1971-01-01,Jackie Stewart,8,75,9.375
1972-01-01,Jackie Stewart,7,60,8.571428571428571
1972-01-01,Niki Lauda,3,7,2.3333333333333335
1973-01-01,Jackie Stewart,13,103,7.923076923076923


Databricks visualization. Run in Databricks to view.

In [0]:
%skip
select concat(driver_name.forename, ' ', driver_name.surname) as driver_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where year between 2011 and 2020
group by driver_name
having count(1) >= 50
order by avg_points desc;

-- select concat(driver_name.forename, ' ', driver_name.surname) as driver_name, 
-- count(1) as total_races,
-- sum(calc_points) as total_points,
-- avg(calc_points) as avg_points
-- from f1_presentation.calc_race_results
-- where year between 2001 and 2010
-- group by driver_name
-- having count(1) >= 50
-- order by avg_points desc



In [0]:
select team_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
group by team_name
having count(1) >= 100
order by avg_points desc


team_name,total_races,total_points,avg_points
Mercedes,385,2931,7.612987012987013
Ferrari,1478,10500,7.104194857916103
McLaren,1032,6865,6.6521317829457365
Red Bull,433,2851,6.584295612009238
Williams,745,4759,6.387919463087249
Benetton,296,1806,6.101351351351352
Team Lotus,352,2100,5.965909090909091
Cooper-Climax,130,772,5.938461538461539
Brabham,235,1373,5.842553191489362
BRM,215,1212,5.637209302325582


In [0]:
create or replace temp view vw_dominant_team
as
select team_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points,
rank() over(order by avg(calc_points) desc) as team_rank
from f1_presentation.calc_race_results
group by team_name
having count(1) >= 100
order by avg_points desc

In [0]:
select to_date(year, 'yyyy') as year, 
team_name,
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where team_name in (select team_name from vw_dominant_team where team_rank <=5 )
group by year, team_name
order by year,avg_points desc

year,team_name,total_races,total_points,avg_points
1950-01-01,Ferrari,8,57,7.125
1951-01-01,Ferrari,22,168,7.636363636363637
1952-01-01,Ferrari,28,202,7.214285714285714
1953-01-01,Ferrari,35,230,6.571428571428571
1954-01-01,Mercedes,12,96,8.0
1954-01-01,Ferrari,25,175,7.0
1955-01-01,Mercedes,15,124,8.266666666666667
1955-01-01,Ferrari,21,131,6.238095238095238
1956-01-01,Ferrari,23,172,7.478260869565218
1957-01-01,Ferrari,20,130,6.5


Databricks visualization. Run in Databricks to view.

In [0]:
select to_date(year, 'yyyy') as year, 
team_name,
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where team_name in (select team_name from vw_dominant_team where team_rank <=5)
group by year, team_name
order by year,avg_points desc

year,team_name,total_races,total_points,avg_points
1950-01-01,Ferrari,8,57,7.125
1951-01-01,Ferrari,22,168,7.636363636363637
1952-01-01,Ferrari,28,202,7.214285714285714
1953-01-01,Ferrari,35,230,6.571428571428571
1954-01-01,Mercedes,12,96,8.0
1954-01-01,Ferrari,25,175,7.0
1955-01-01,Mercedes,15,124,8.266666666666667
1955-01-01,Ferrari,21,131,6.238095238095238
1956-01-01,Ferrari,23,172,7.478260869565218
1957-01-01,Ferrari,20,130,6.5


Databricks visualization. Run in Databricks to view.

In [0]:
%skip
select team_name, 
count(1) as total_races,
sum(calc_points) as total_points,
avg(calc_points) as avg_points
from f1_presentation.calc_race_results
where year between 2011 and 2020
group by team_name
having count(1) >= 100
order by avg_points desc
